In [ ]:
import torch
import torch.nn as nn
import optuna

from lib.glob_fix import glob
from tqdm import tqdm

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split

from lib.models import BaseLearnerRegression, cyclic_cosine_annealing_lr

In [ ]:
N_SPLITS = 3

In [ ]:
def objective(trial):
    # Hyperparameters to optimize: 2 hidden layers, 1e-3 < lr < 1e-1, 50 < epochs < 100
    loss_function = nn.MSELoss()
    models = {}
    for i in range(4):
        models[f'm{i}_'] = {
            'hidden_sizes': [trial.suggest_categorical(f'm{i}_n_units_l1', [2 ** i for i in range(4, 9)]), trial.suggest_categorical(f'm{i}_n_units_l2', [2 ** i for i in range(4, 9)])],
            'activation': trial.suggest_categorical(f'm{i}_activation', ['relu', 'sigmoid', 'tanh']),
            'lr': trial.suggest_float(f'm{i}_lr', 1e-3, 1e-1),
            'epochs': 10 #trial.suggest_int(f'm{i}_epochs', 50, 100) # We have since fixed the #epochs to 10 due to snapshot
        }

    # Load the data
    global_loss = 0
    trial_model_selectors = []
    for fname in glob('data/train-datasets/*.csv'):
        df = pd.read_csv(fname)

        # Divide the data into 3 folds
        kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

        # Define the hyperparameters to optimize
        fold_id = 0
        for train_idx, test_idx in kf.split(df):
            train_set = df.iloc[train_idx]

            input_size = train_set.shape[1] - 1

            losses = []
            for mk in models.keys():
                model = BaseLearnerRegression(input_size, models[mk]['hidden_sizes'], models[mk]['activation'], loss_function, lr=models[mk]['lr'])
                model.setup_data_loaders(train_set, validation_size=0.3)
                for epoch in range(models[mk]['epochs']):
                    lr_update = cyclic_cosine_annealing_lr(model.lr, 10, 0, epoch)
                    model.train(1, lr_update=lr_update)
                val_loss = model.evaluate()
                losses.append(val_loss)

            trial_model_selectors.append({'dataset': fname, 'fold': fold_id, 'model': f'm{np.argmin(losses)}_'})

            global_loss += np.min(losses)
            fold_id += 1

    # Save the model selectors
    pd.DataFrame(trial_model_selectors).to_csv(f'results/trials/{trial.number}.csv', index=False)

    return global_loss

In [ ]:
N_SPLITS = 3
N_TRIALS = 100

# Define the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=N_TRIALS)
study.trials_dataframe().to_csv('results/parameter_search_results.csv', index=False)